In [1]:
# This notebook will mount an azure storage blob and read a file from a container

# Start by specifying your storage blob information

container = "<your container name>"
storageAccount = "<your storage account name> "  
accessKey = "<your storage account key>"
accountKey="fs.azure.account.key.{}.blob.core.windows.net".format(storageAccount)

#print(accountKey)
#print(accessKey)
#print(storageAccount)

# Set the credentials to Spark configuration
spark.conf.set(accountKey, accessKey)

In [2]:
# This will create a mount point with Databricks FS protocol and request files using a file path
# like:  dbfs:/mnt/<container name>/<partial path>

inputSource = "wasbs://{}@{}.blob.core.windows.net".format(container, storageAccount)
mountPoint = "/mnt/" + container
extraConfig = {accountKey: accessKey}

print("Mounting: {}".format(mountPoint))
print(inputSource)
print(mountPoint)
print(extraConfig)

try:
  dbutils.fs.mount(source = inputSource, mount_point = str(mountPoint),  extra_configs = extraConfig )
  print("=> Succeeded")
except Exception as e:
  if "Directory already mounted" in str(e):
    print("=> Directory {} already mounted".format(mountPoint))
  else:
    raise(e)

In [3]:
# To test the storage mount works, read a file from the container and print it out

# Open and read the file from mounted storage
print(container)
inputFilePath = "/dbfs/mnt/{}/{}".format(container, 'usd_to_eur.csv') 
file = open(inputFilePath, 'r')
print(file.read())


In [4]:
# Load the data into a dataframe and run some statistics on the data
inputFilePath = "dbfs:/mnt/{}/{}".format(container, "usd_to_eur.csv") 
df = spark.read.format('csv').load(inputFilePath, header=True, inferSchema=True) 
df.printSchema() 
df.describe().show() 
df.head(5)
